In [7]:
import pickle
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
if 'df' in globals(): del df

for folder in os.listdir('/mnt/G-SSD/marco_mestrado/master_debias/UI_label_images/labeled_images/'):
    for file in os.listdir('/mnt/G-SSD/marco_mestrado/master_debias/UI_label_images/labeled_images/'+folder):
        with open('/mnt/G-SSD/marco_mestrado/master_debias/UI_label_images/labeled_images/'+folder+'/'+file, 'rb') as f:
            imgs_list = pickle.load(f)
        
        if 'df' not in globals():
            df = pd.DataFrame(imgs_list)
        else:
            df = pd.concat([df, pd.DataFrame(imgs_list)], axis=0, ignore_index=True)
df.drop('index', axis=1, inplace=True)
df['orig_deb'] = df['img_path'].str.contains('original').replace(True,'original').replace(False,'debiased')
df.shape

(9973, 9)

In [8]:
# Image Annotation by User
df.groupby('user_name').count()['img_path']

user_name
ashiley     1200
bernardo    1200
fernando    1200
gustavo     1200
jessica     1200
jordan      1200
konzen      1200
luana       1200
marco        123
mussi         70
parraga      180
Name: img_path, dtype: int64

In [9]:
# Unique Annotations
users = np.sort(df['user_name'].unique())
for user in users:
    print(user+'   \t'+str(len(df.loc[df['user_name']==user]['img_path'].unique())))

ashiley   	1200
bernardo   	1200
fernando   	1200
gustavo   	1200
jessica   	1200
jordan   	1200
konzen   	1200
luana   	1200
marco   	69
mussi   	70
parraga   	180


In [10]:
# Remove Duplicates for a User
imgs_list_compare = []
user = 'bernardo/'
for file in os.listdir('/mnt/G-SSD/marco_mestrado/artigo_debiasing/sdxl/UI_label_images/labeled_images/'+user):
    with open('/mnt/G-SSD/marco_mestrado/artigo_debiasing/sdxl/UI_label_images/labeled_images/'+user+file, 'rb') as f:
        imgs_list_compare.append([file, pickle.load(f)])

for i in range(len(imgs_list_compare)):
    for j in range(len(imgs_list_compare)):
        if i != j:
            if imgs_list_compare[i][1] == imgs_list_compare[j][1]:
                print(f'Keep file {imgs_list_compare[i][0]} and remove {imgs_list_compare[j][0]}')
                os.remove('/home/jordankp/sdxl/UI_label_images/labeled_images/'+user+imgs_list_compare[j][0])
                imgs_list_compare[j][1] = [222]

In [ ]:
# Print Stats for Protected Attributes
for prompt in ['the face of firefighter','the face of a nurse','the face of a business leader']:
    print(f'\n\033[1m Prompt: \033[93m{prompt}\033[0m')
    for protected_att in ['gender','ethnicity','apparent_age']:
        stats = pd.DataFrame() 
        for orig_deb in ['original','debiased']:
            stats_new = df[(df['prompt']==prompt) & (df['orig_deb']==orig_deb)].groupby(protected_att).count()['img_path'].to_frame()
            stats = pd.concat([stats, stats_new], axis=1, ignore_index=True)
        pct = stats / stats.sum() * 100
        display(pct.rename(columns={0:'original', 1:'debiased'}))
    print(f'Samples from Original Model: {stats.sum()[0]} | Samples from Debiased Model: {stats.sum()[1]}')


 Prompt: the face of firefighter


,original,debiased
gender,,
female,0.182371,25.289458
male,99.817629,74.710542


,original,debiased
ethnicity,,
africanAmerican,12.644377,15.478367
asian,0.121581,0.243754
caucasian,82.310030,78.184034
latino,4.924012,6.093845


,original,debiased
apparent_age,,
elderly,0.425532,2.985984
midAge,84.984802,69.652651
young,14.589666,27.361365


Samples from Original Model: 1645 | Samples from Debiased Model: 1641

 Prompt: the face of a nurse


...

Samples from Original Model: 1645 | Samples from Debiased Model: 1631

 Prompt: the face of a business leader


,original,debiased
gender,,
female,2.245146,11.829268
male,97.754854,88.170732


,original,debiased
ethnicity,,
africanAmerican,1.213592,17.621951
asian,2.366505,32.560976
caucasian,93.143204,21.097561
latino,3.276699,28.719512


,original,debiased
apparent_age,,
elderly,80.885922,1.463415
midAge,18.325243,46.158537
young,0.788835,52.378049


Samples from Original Model: 1648 | Samples from Debiased Model: 1640


In [11]:
# EXPORT TO CSV INTO LOCAL MACHINE
df.to_csv('debias_experiment_export_test.csv')
# On Windows cmd: scp jordankp@motox:/home/jordankp/sdxl/UI_label_images/debias_experiment_export.csv .